In [1]:
import os
import h5py
import time
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from pandas import DataFrame
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.ticker as ticker
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

#import package for Gaussian Process Regression (GPR)
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.preprocessing import StandardScaler
# import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.layers import Input, Dense, Activation,Dropout
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers import Conv1D, MaxPooling1D, LSTM, Bidirectional
from keras import backend as K
### Set-up - Define file location
filename = 'N-CMAPSS_DS04.h5'

# Load data
with h5py.File(filename, 'r') as hdf:
        # Development set
        W_dev = np.array(hdf.get('W_dev'))             # W
        X_s_dev = np.array(hdf.get('X_s_dev'))         # X_s
        X_v_dev = np.array(hdf.get('X_v_dev'))         # X_v
        T_dev = np.array(hdf.get('T_dev'))             # T
        Y_dev = np.array(hdf.get('Y_dev'))             # RUL  
        A_dev = np.array(hdf.get('A_dev'))             # Auxiliary

        # Test set
        W_test = np.array(hdf.get('W_test'))           # W
        X_s_test = np.array(hdf.get('X_s_test'))       # X_s
        X_v_test = np.array(hdf.get('X_v_test'))       # X_v
        T_test = np.array(hdf.get('T_test'))           # T
        Y_test = np.array(hdf.get('Y_test'))           # RUL  
        A_test = np.array(hdf.get('A_test'))           # Auxiliary
        
        # Varnams
        W_var = np.array(hdf.get('W_var'))
        X_s_var = np.array(hdf.get('X_s_var'))  
        X_v_var = np.array(hdf.get('X_v_var')) 
        T_var = np.array(hdf.get('T_var'))
        A_var = np.array(hdf.get('A_var'))
        
        # from np.array to list dtype U4/U5
        W_var = list(np.array(W_var, dtype='U20'))
        X_s_var = list(np.array(X_s_var, dtype='U20'))  
        X_v_var = list(np.array(X_v_var, dtype='U20')) 
        T_var = list(np.array(T_var, dtype='U20'))
        A_var = list(np.array(A_var, dtype='U20'))
                          
W = np.concatenate((W_dev, W_test), axis=0)  
X_s = np.concatenate((X_s_dev, X_s_test), axis=0)
X_v = np.concatenate((X_v_dev, X_v_test), axis=0)
T = np.concatenate((T_dev, T_test), axis=0)
Y = np.concatenate((Y_dev, Y_test), axis=0) 
A = np.concatenate((A_dev, A_test), axis=0) 

# Create DataFrames with the specified columns
df_A_dev = DataFrame(data=A_dev, columns=A_var)
df_W_dev = DataFrame(data=W_dev, columns=W_var)
df_T_dev = DataFrame(data=T_dev, columns=T_var)
df_X_s_dev = DataFrame(data=X_s_dev, columns=X_s_var)
df_X_v_dev = DataFrame(data=X_v_dev, columns=X_v_var)

# Separate features (X) and target variable (y)
df_A_test = DataFrame(data=A_test, columns=A_var)
df_W_test = DataFrame(data=W_test, columns=W_var)
df_T_test = DataFrame(data=T_test, columns=T_var)
df_X_s_test = DataFrame(data=X_s_test, columns=X_s_var)
df_X_v_test = DataFrame(data=X_v_test, columns=X_v_var)



In [2]:
# Add columns to the dataframes: unit, cycle and Fc
df_T_dev['unit'] = df_A_dev['unit'].values
df_T_dev['cycle'] = df_A_dev['cycle'].values
df_T_dev['Fc'] = df_A_dev['Fc'].values
df_T_dev['hs'] = df_A_dev['hs'].values

df_W_dev['unit'] = df_A_dev['unit'].values
df_W_dev['cycle'] = df_A_dev['cycle'].values
df_W_dev['Fc'] = df_A_dev['Fc'].values
df_W_dev['hs'] = df_A_dev['hs'].values


df_T_test['unit'] = df_A_test['unit'].values
df_T_test['cycle'] = df_A_test['cycle'].values
df_T_test['Fc'] = df_A_test['Fc'].values
df_T_test['hs'] = df_A_test['hs'].values

df_W_test['unit'] = df_A_test['unit'].values
df_W_test['cycle'] = df_A_test['cycle'].values
df_W_test['Fc'] = df_A_test['Fc'].values
df_W_test['hs'] = df_A_test['hs'].values

df_X_s_dev['unit'] = df_A_dev['unit'].values
df_X_s_dev['cycle'] = df_A_dev['cycle'].values
df_X_s_dev['Fc'] = df_A_dev['Fc'].values
df_X_s_dev['hs'] = df_A_dev['hs'].values

df_X_v_dev['unit'] = df_A_dev['unit'].values
df_X_v_dev['cycle'] = df_A_dev['cycle'].values
df_X_v_dev['Fc'] = df_A_dev['Fc'].values
df_X_v_dev['hs'] = df_A_dev['hs'].values

df_X_s_test['unit'] = df_A_test['unit'].values
df_X_s_test['cycle'] = df_A_test['cycle'].values
df_X_s_test['Fc'] = df_A_test['Fc'].values
df_X_s_test['hs'] = df_A_test['hs'].values

df_X_v_test['unit'] = df_A_test['unit'].values
df_X_v_test['cycle'] = df_A_test['cycle'].values
df_X_v_test['Fc'] = df_A_test['Fc'].values
df_X_v_test['hs'] = df_A_test['hs'].values

T_map_train = df_T_dev[['unit','cycle', 'Fc','hs','fan_eff_mod', 'fan_flow_mod', 'LPC_eff_mod', 'LPC_flow_mod', 'HPC_eff_mod', 'HPC_flow_mod', 'HPT_eff_mod', 'HPT_flow_mod', 'LPT_eff_mod', 'LPT_flow_mod']]
A_map_train = df_A_dev[['unit', 'cycle', 'Fc','hs']]
W_map_train = df_W_dev[['unit', 'cycle', 'Fc','hs','alt', 'Mach', 'TRA','T2']]
X_s_map_train = df_X_s_dev[['unit', 'cycle', 'Fc','hs','T48', 'T50', 'P50', 'Wf', 'Nf', 'Nc', 'T24', 'T30', 'P15', 'P21', 'P24', 'Ps30', 'P40', 'P2']]
X_v_map_train = df_X_v_dev[['unit', 'cycle', 'Fc','hs','T40', 'P30', 'P45', 'W21', 'W22', 'W25', 'W31', 'W32', 'W48', 'W50', 'SmFan', 'SmLPC', 'SmHPC', 'phi']]

T_map_test = df_T_test[['unit','cycle', 'Fc', 'hs','fan_eff_mod', 'fan_flow_mod', 'LPC_eff_mod', 'LPC_flow_mod', 'HPC_eff_mod', 'HPC_flow_mod', 'HPT_eff_mod', 'HPT_flow_mod', 'LPT_eff_mod', 'LPT_flow_mod']]
A_map_test = df_A_test[['unit', 'cycle', 'Fc', 'hs']]
W_map_test = df_W_test[['unit', 'cycle', 'Fc', 'hs','alt', 'Mach', 'TRA','T2']]
X_s_map_test = df_X_s_test[['unit', 'cycle', 'Fc', 'hs','T48', 'T50', 'P50', 'Wf', 'Nf', 'Nc', 'T24', 'T30', 'P15', 'P21', 'P24', 'Ps30', 'P40', 'P2']]
X_v_map_test = df_X_v_test[['unit', 'cycle', 'Fc', 'hs','T40', 'P30', 'P45', 'W21', 'W22', 'W25', 'W31', 'W32', 'W48', 'W50', 'SmFan', 'SmLPC', 'SmHPC', 'phi']]

In [3]:
# Calculate the 3rd quartile values for each unique combination of unit, cycle, and Fc values in the training set
df_T_map_1q_train = T_map_train.groupby(['unit', 'cycle', 'Fc','hs']).quantile(0.75).reset_index()
df_W_map_1q_train = W_map_train.groupby(['unit', 'cycle', 'Fc','hs']).quantile(0.75).reset_index()
df_A_map_1q_train = A_map_train.groupby(['unit', 'cycle', 'Fc','hs']).quantile(0.75).reset_index()
df_X_s_map_1q_train = X_s_map_train.groupby(['unit', 'cycle', 'Fc','hs']).quantile(0.75).reset_index()
df_X_v_map_1q_train = X_v_map_train.groupby(['unit', 'cycle', 'Fc','hs']).quantile(0.75).reset_index()

# Calculate the 3rd qartile values for each unique combination of unit, cycle, and Fc values in the test set
df_T_map_1q_test = T_map_test.groupby(['unit', 'cycle', 'Fc','hs']).quantile(0.75).reset_index()
df_W_map_1q_test = W_map_test.groupby(['unit', 'cycle', 'Fc','hs']).quantile(0.75).reset_index()
df_A_map_1q_test = A_map_test.groupby(['unit', 'cycle', 'Fc','hs']).quantile(0.75).reset_index()
df_X_s_map_1q_test = X_s_map_test.groupby(['unit', 'cycle', 'Fc','hs']).quantile(0.75).reset_index()
df_X_v_map_1q_test = X_v_map_test.groupby(['unit', 'cycle', 'Fc','hs']).quantile(0.75).reset_index()


In [4]:
df_T_map_1q_train.describe()

,unit,cycle,Fc,hs,fan_eff_mod,fan_flow_mod,LPC_eff_mod,LPC_flow_mod,HPC_eff_mod,HPC_flow_mod,HPT_eff_mod,HPT_flow_mod,LPT_eff_mod,LPT_flow_mod
count,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.0,512.0,512.0,512.0,512.0,512.0,512.0,512.0
mean,3.529297,44.003906,2.634766,0.205078,-0.011870,-0.010126,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,1.709625,26.054662,0.481967,0.404154,0.024338,0.016309,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,1.000000,1.000000,2.000000,0.000000,-0.223446,-0.121209,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,2.000000,22.000000,2.000000,0.000000,-0.012202,-0.011992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,3.000000,43.000000,3.000000,0.000000,-0.002929,-0.003303,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,5.000000,64.250000,3.000000,0.000000,-0.000866,-0.001051,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,6.000000,100.000000,3.000000,1.000000,0.000027,-0.000150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
T_map_for_RUL = pd.concat([df_T_map_1q_train, df_T_map_1q_test], axis=0)
W_map_for_RUL = pd.concat([df_W_map_1q_train, df_W_map_1q_test], axis=0)
A_map_for_RUL = pd.concat([df_A_map_1q_train, df_A_map_1q_test], axis=0)
X_s_map_for_RUL = pd.concat([df_X_s_map_1q_train, df_X_s_map_1q_test], axis=0)
X_v_map_for_RUL = pd.concat([df_X_v_map_1q_train, df_X_v_map_1q_test], axis=0)

T_map_for_RUL = T_map_for_RUL.reset_index(drop=True)
W_map_for_RUL = W_map_for_RUL.reset_index(drop=True)
A_map_for_RUL = A_map_for_RUL.reset_index(drop=True)
X_s_map_for_RUL = X_s_map_for_RUL.reset_index(drop=True)
X_v_map_for_RUL = X_v_map_for_RUL.reset_index(drop=True)

# Create DataFrames with the specified columns
df_T_map_for_RUL = DataFrame(data=T_map_for_RUL, columns=['fan_eff_mod', 'fan_flow_mod', 'LPC_eff_mod', 'LPC_flow_mod', 'HPC_eff_mod', 'HPC_flow_mod', 'HPT_eff_mod', 'HPT_flow_mod', 'LPT_eff_mod', 'LPT_flow_mod'])
df_W_map_for_RUL = DataFrame(data=W_map_for_RUL, columns=['alt', 'Mach', 'TRA','T2'])
df_A_map_for_RUL = DataFrame(data=A_map_for_RUL, columns=A_map_for_RUL.columns)
df_X_s_map_for_RUL = DataFrame(data=X_s_map_for_RUL, columns=['T48', 'T50', 'P50', 'Wf', 'Nf', 'Nc', 'T24', 'T30', 'P15', 'P21', 'P24', 'Ps30', 'P40', 'P2'])
df_X_v_map_for_RUL = DataFrame(data=X_v_map_for_RUL, columns=['T40', 'P30', 'P45', 'W21', 'W22', 'W25', 'W31', 'W32', 'W48', 'W50', 'SmFan', 'SmLPC', 'SmHPC', 'phi'])

In [6]:
df_W_map_for_RUL.describe()

,alt,Mach,TRA,T2
count,856.000000,856.000000,856.000000,856.000000
mean,22855.847839,0.636379,75.053901,502.573385
std,4737.964752,0.055576,6.305633,5.672468
min,11693.000000,0.471807,55.458588,483.240022
25%,19996.000000,0.602595,71.987404,499.637972
50%,22953.250000,0.639009,77.255310,503.260514
75%,26991.000000,0.675943,79.628342,506.347003
max,32978.000000,0.734958,86.044312,517.620690


In [7]:
# Load predictions from degradation 
y_pred = pd.read_csv('tetha_train_DS004.csv')
y_1_pred = pd.read_csv('tetha_test_DS004.csv')

In [8]:
y_1_pred.describe()

,fan_eff_mod,fan_flow_mod,LPC_eff_mod,LPC_flow_mod,HPC_eff_mod,HPC_flow_mod,HPT_eff_mod,HPT_flow_mod,LPT_eff_mod,LPT_flow_mod
count,344.000000,344.000000,344.0,344.0,344.0,344.0,344.0,344.0,344.0,344.0
mean,-0.010115,-0.008321,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.017832,0.012303,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,-0.129423,-0.066793,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,-0.011306,-0.009875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,-0.002935,-0.002785,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,-0.001084,-0.000927,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,-0.000309,-0.000116,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:

tetha_infered = pd.DataFrame(data = y_pred , columns=['fan_eff_mod', 'fan_flow_mod', 'LPC_eff_mod', 'LPC_flow_mod', 'HPC_eff_mod', 'HPC_flow_mod', 'HPT_eff_mod', 'HPT_flow_mod', 'LPT_eff_mod', 'LPT_flow_mod'])
tetha_infered_1 = pd.DataFrame(data=y_1_pred, columns=['fan_eff_mod', 'fan_flow_mod', 'LPC_eff_mod', 'LPC_flow_mod', 'HPC_eff_mod', 'HPC_flow_mod', 'HPT_eff_mod', 'HPT_flow_mod', 'LPT_eff_mod', 'LPT_flow_mod'])
tetha_infered = tetha_infered.reset_index(drop=True)
tetha_infered_1 = tetha_infered_1.reset_index(drop=True)

tetha_for_RUL = pd.concat([tetha_infered, tetha_infered_1], axis=0)
tetha_for_RUL = tetha_for_RUL.reset_index(drop=True)
tetha_for_RUL.describe()

,fan_eff_mod,fan_flow_mod,LPC_eff_mod,LPC_flow_mod,HPC_eff_mod,HPC_flow_mod,HPT_eff_mod,HPT_flow_mod,LPT_eff_mod,LPT_flow_mod
count,856.000000,856.000000,856.0,856.0,856.0,856.0,856.0,856.0,856.0,856.0
mean,-0.011163,-0.009399,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.021830,0.014795,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,-0.204706,-0.113627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,-0.011739,-0.011076,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,-0.002871,-0.003034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,-0.000952,-0.000990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,0.000022,-0.000116,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# compile my data in x and y for an RUL inference

# For x data, I'm gonna use W, T, A, X_s, X_v, tetha_infered

x_for_RUL = pd.concat([df_W_map_for_RUL, df_A_map_for_RUL, df_X_s_map_for_RUL, df_X_v_map_for_RUL, tetha_for_RUL], axis=1)
x_for_RUL = x_for_RUL.reset_index(drop=True)

# Get the maximum cycle number for each unit
max_cycle = df_A_map_for_RUL.groupby('unit')['cycle'].max()

# Calculate the RUL for each row
df_A_map_for_RUL['RUL'] = df_A_map_for_RUL.groupby('unit')['cycle'].transform('max') - df_A_map_for_RUL['cycle']

# Add the RUL column to the y data
y_for_RUL = df_A_map_for_RUL['RUL']

In [53]:
# Split my data into training and testing sets

x_for_RUL_1 = x_for_RUL.values
y_for_RUL_1 = y_for_RUL.values

# Calculate the index at which to split the data
split_idx = int(len(x_for_RUL) * 0.901)

# Split the data
x_train = x_for_RUL[:split_idx]
x_test = x_for_RUL[split_idx:]
y_train = y_for_RUL[:split_idx]
y_test = y_for_RUL[split_idx:]


In [54]:
X_train_for_RUL = pd.DataFrame(data=x_train, columns=x_for_RUL.columns)
X_test_for_RUL = pd.DataFrame(data=x_test, columns=x_for_RUL.columns)
y_train_for_RUL = pd.DataFrame(data=y_train, columns=['RUL'])
y_test_for_RUL = pd.DataFrame(data=y_test, columns=['RUL'])


In [55]:
# plot x_train and y_train
plt.plot(y_train_for_RUL)
plt.ylabel('RUL')
plt.xlabel('Time')
plt.show()


In [56]:
# plot x_test and y_test
plt.plot(y_test_for_RUL)
plt.ylabel('RUL')
plt.xlabel('Time')
plt.show()


In [57]:
# Save Dataframes to CSV
X_train_for_RUL.to_csv('X_train_for_RUL_DS004.csv', index = False)
y_train_for_RUL.to_csv('y_for_RUL_DS004.csv', index = False)
X_test_for_RUL.to_csv('X_test_for_RUL_DS004.csv', index = False)
y_test_for_RUL.to_csv('y_test_for_RUL_DS004.csv', index = False)

In [58]:
lets_for_x = pd.read_csv('X_train_for_RUL_DS004.csv')
lets_for_x.describe()

,alt,Mach,TRA,T2,unit,cycle,Fc,hs,T48,T50,...,fan_eff_mod,fan_flow_mod,LPC_eff_mod,LPC_flow_mod,HPC_eff_mod,HPC_flow_mod,HPT_eff_mod,HPT_flow_mod,LPT_eff_mod,LPT_flow_mod
count,771.000000,771.000000,771.000000,771.000000,771.000000,771.000000,771.000000,771.000000,771.000000,771.000000,...,771.000000,771.000000,771.0,771.0,771.0,771.0,771.0,771.0,771.0,771.0
mean,22499.830091,0.632642,74.865351,502.940347,5.012970,44.110246,2.421530,0.217899,1709.168556,1156.072230,...,-0.011616,-0.009331,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,4660.811898,0.055321,6.369031,5.581671,2.550241,26.040718,0.494125,0.413086,41.020206,20.174663,...,0.022783,0.015028,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,11693.000000,0.471807,55.458588,484.811972,1.000000,1.000000,2.000000,0.000000,1605.306962,1097.109905,...,-0.204706,-0.113627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,19991.000000,0.599933,71.718239,500.295405,3.000000,22.000000,2.000000,0.000000,1675.633319,1142.206172,...,-0.011914,-0.010682,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,22359.500000,0.637497,77.167419,503.547954,5.000000,43.000000,2.000000,0.000000,1708.387381,1156.681827,...,-0.002866,-0.002959,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,25812.875000,0.670572,79.452560,506.572014,7.000000,65.000000,3.000000,0.000000,1739.393656,1170.649283,...,-0.000931,-0.000950,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,32978.000000,0.734958,86.044312,517.620690,9.000000,100.000000,3.000000,1.000000,1807.221815,1213.429331,...,0.000022,-0.000116,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
